In [52]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data Analysis

### Awards Player

In [53]:
awards_players_dataSet = pd.read_csv('../Other/basketballPlayoffs/awards_players.csv')
#print(awards_players_dataSet.info())
columns_to_remove = ['lgID']
awards_players_new = awards_players_dataSet.drop(columns=columns_to_remove)

awards_players_new = awards_players_new[(awards_players_new['award'] != 'Kim Perrot Sportsmanship Award') & (awards_players_new['award'] != 'Sixth Woman of the Year') ]

awards_players_new.to_csv('newDataset/awards_players_cleaned.csv', index=False)

### Coaches

In [54]:
coaches_dataSet = pd.read_csv('../Other/basketballPlayoffs/coaches.csv')
#print(coaches_dataSet.info())

#Check if stint could be removed
columns_to_remove = ['lgID', 'stint']
coaches_new = coaches_dataSet.drop(columns=columns_to_remove)

coaches_new.to_csv('newDataset/coaches_cleaned.csv', index=False)

### Players

In [55]:
players_dataSet = pd.read_csv('../Other/basketballPlayoffs/players.csv')
#print(players_dataSet.info())

#Maybe everything is usless
columns_to_remove = ['firstseason', 'lastseason', 'height', 'weight', 'college', 'collegeOther', 'deathDate']
players_dataSet_new = players_dataSet.drop(columns=columns_to_remove)

players_dataSet.to_csv('newDataset/players_cleaned.csv', index=False)

### Players Teams

In [56]:
players_teams_dataSet = pd.read_csv('../Other/basketballPlayoffs/players_teams.csv')
#print(players_teams_dataSet.info())

#Check if something else could be removed
columns_to_remove = ['lgID', 'turnovers']
players_teams_new = players_teams_dataSet.drop(columns=columns_to_remove)

players_teams_new.to_csv('newDataset/players_teams_cleaned.csv', index=False)

### Series Post

In [57]:
series_post_dataSet = pd.read_csv('../Other/basketballPlayoffs/series_post.csv')
#print(series_post_dataSet.info())

#Check if something else could be removed
columns_to_remove = ['lgIDWinner', 'lgIDLoser']
series_post_new = series_post_dataSet.drop(columns=columns_to_remove)

series_post_new.to_csv('newDataset/series_post_cleaned.csv', index=False)

### Teams

In [58]:
teams_dataSet = pd.read_csv('../Other/basketballPlayoffs/teams.csv')
#print(teams_dataSet.info()) 
columns_to_remove = ['lgID','divID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB', 'GP', 'min', 'attend', 'arena'] 
teams_new = teams_dataSet.drop(columns=columns_to_remove)
teams_new.to_csv('newDataset/teams_cleaned.csv', index=False)

### Teams Posts

In [59]:
# Nothing to do for now
teams_post_dataSet = pd.read_csv('../Other/basketballPlayoffs/teams_post.csv')
columns_to_remove = [] 
teams_post_new = teams_post_dataSet.drop(columns=columns_to_remove)
teams_post_new.to_csv('newDataset/teams_post.csv', index=False)